# direct marketing data set demo

In [1]:
import numpy as np                                # For matrix operations and numerical processing
import pandas as pd                               # For munging tabular data
import matplotlib.pyplot as plt                   # For charts and visualizations
from IPython.display import Image                 # For displaying images in the notebook
from IPython.display import display               # For displaying outputs in the notebook
from time import gmtime, strftime                 # For labeling SageMaker models, endpoints, etc.
import sys                                        # For writing outputs to notebook
import math                                       # For ceiling function
import json                                       # For parsing hosting outputs
import os                                         # For manipulating filepath names
import sagemaker 
import zipfile 
import boto3

In [2]:
!wget https://sagemaker-sample-data-us-west-2.s3-us-west-2.amazonaws.com/autopilot/direct_marketing/bank-additional.zip

with zipfile.ZipFile('bank-additional.zip', 'r') as zip_ref:
    zip_ref.extractall('.')

--2021-09-02 08:34:07--  https://sagemaker-sample-data-us-west-2.s3-us-west-2.amazonaws.com/autopilot/direct_marketing/bank-additional.zip
Resolving sagemaker-sample-data-us-west-2.s3-us-west-2.amazonaws.com (sagemaker-sample-data-us-west-2.s3-us-west-2.amazonaws.com)... 52.218.236.73
Connecting to sagemaker-sample-data-us-west-2.s3-us-west-2.amazonaws.com (sagemaker-sample-data-us-west-2.s3-us-west-2.amazonaws.com)|52.218.236.73|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 432828 (423K) [application/zip]
Saving to: ‘bank-additional.zip.2’

bank-additional.zip 100%[===================>] 422.68K   751KB/s    in 0.6s    

2021-09-02 08:34:08 (751 KB/s) - ‘bank-additional.zip.2’ saved [432828/432828]



In [3]:
data = pd.read_csv('./bank-additional/bank-additional-full.csv')
pd.set_option('display.max_columns', 500)     # Make sure we can see all of the columns
pd.set_option('display.max_rows', 20)         # Keep the output on one page
data

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,duration,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,56,housemaid,married,basic.4y,no,no,no,telephone,may,mon,261,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
1,57,services,married,high.school,unknown,no,no,telephone,may,mon,149,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
2,37,services,married,high.school,no,yes,no,telephone,may,mon,226,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
3,40,admin.,married,basic.6y,no,no,no,telephone,may,mon,151,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
4,56,services,married,high.school,no,no,yes,telephone,may,mon,307,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41183,73,retired,married,professional.course,no,yes,no,cellular,nov,fri,334,1,999,0,nonexistent,-1.1,94.767,-50.8,1.028,4963.6,yes
41184,46,blue-collar,married,professional.course,no,no,no,cellular,nov,fri,383,1,999,0,nonexistent,-1.1,94.767,-50.8,1.028,4963.6,no
41185,56,retired,married,university.degree,no,yes,no,cellular,nov,fri,189,2,999,0,nonexistent,-1.1,94.767,-50.8,1.028,4963.6,no
41186,44,technician,married,professional.course,no,no,no,cellular,nov,fri,442,1,999,0,nonexistent,-1.1,94.767,-50.8,1.028,4963.6,yes


In [4]:
# store the file in a bucket: 
import sagemaker
bucket=sagemaker.Session().default_bucket()
prefix = 'sagemaker/bank-additional'
region=boto3.Session().region_name

In [5]:
bucket

'sagemaker-ap-southeast-2-153805901131'

In [6]:
boto3.Session().resource('s3').Bucket(bucket).Object(os.path.join(prefix, 'bank-additional-full.csv')).upload_file('./bank-additional/bank-additional-full.csv')

In [7]:
data_file_uri = 's3://sagemaker-ap-southeast-2-153805901131/sagemaker/bank-additional/bank-additional-full.csv'

## Query S3 data via Athena

In [8]:
from IPython.display import display, HTML
import time

In [9]:
def athena_create_table(query_file, wait=None):
    create_table_uri = athena_execute(query_file, 'txt', wait)
    return create_table_uri

def athena_query_table(query_str, wait=None):
    results_uri = athena_execute(query_str, 'csv', wait)
    return results_uri

console_s3_uri= 'https://s3.console.aws.amazon.com/s3/object/'

def athena_execute(query_str, ext, wait):
    display(HTML(f'Executing query:<br><br><code>{query_str}</code><br><br>'))
        
    athena = boto3.client('athena')
    s3_dest = f's3://{bucket}/athena/results/'
    query_id = athena.start_query_execution(
        QueryString= query_str, 
         ResultConfiguration={'OutputLocation': s3_dest}
    )['QueryExecutionId']
        
    results_uri = f'{s3_dest}{query_id}.{ext}'
        
    start = time.time()
    while wait == None or wait == 0 or time.time() - start < wait:
        result = athena.get_query_execution(QueryExecutionId=query_id)
        status = result['QueryExecution']['Status']['State']
        if wait == 0 or status == 'SUCCEEDED':
            break
        elif status in ['QUEUED','RUNNING']:
            continue
        else:
            raise Exception(f"query {query_id} failed with status {status} - {result['QueryExecution']['Status']['StateChangeReason']}")

        time.sleep(3) 

    console_url = f'{console_s3_uri}{bucket}/athena/results/{query_id}.{ext}?region={region}&tab=overview'
    display(HTML(f'results are located at <a target="_blank" href="{console_url}">{results_uri}</a>'))
    
    return results_uri

In [10]:
ddl_file = "./data/direct-marketing.ddl"
with open(ddl_file) as f:
    query_str = f.read()  
    query = query_str % f's3://{bucket}/{prefix}/'
    athena_create_table(query)

In [11]:
athena_query = "SELECT * FROM directmarketing"
result_uri = athena_query_table(athena_query)

In [12]:
s3 = boto3.client('s3')
result_file_suffix = 'athena/results/3751ae9d-785e-4ff8-b9b4-a14c62791b99.csv' # please update the suffix with the result file
s3.download_file(bucket, result_file_suffix, './result.csv')

In [13]:
import pandas as pd

In [14]:
df = pd.read_csv('./result.csv')
df.head()

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,duration,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,56,housemaid,married,basic.4y,no,no,no,telephone,may,mon,261,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
1,57,services,married,high.school,unknown,no,no,telephone,may,mon,149,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
2,37,services,married,high.school,no,yes,no,telephone,may,mon,226,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
3,40,admin.,married,basic.6y,no,no,no,telephone,may,mon,151,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
4,56,services,married,high.school,no,no,yes,telephone,may,mon,307,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
